In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [5]:
df = pd.read_csv('test_1_calorie.csv')

In [6]:
df

,Duration,Date,Pulse,Maxpulse,Calories
0,60,2020/12/01,110.0,130.0,409.1
1,60,2020/12/02,117.0,145.0,479.0
2,60,2020/12/03,103.0,135.0,340.0
3,45,2020/12/04,109.0,175.0,282.4
4,45,2020/12/05,117.0,148.0,406.0
...,...,...,...,...,...
170,60,18.05.2021,104.0,131.0,350.0
171,90,19th of May 2021,107.0,138.0,380.0
172,45,May-20-21,95.0,121.0,240.5
173,60,21 May 21,102.0,128.0,305.8


In [7]:
# Display the original data types
print("Original data types:")
print(df.dtypes)
print("\nFirst few rows before conversion:")
print(df.head())

Original data types:
Duration      int64
Date         object
Pulse       float64
Maxpulse    float64
Calories    float64
dtype: object

First few rows before conversion:
   Duration        Date  Pulse  Maxpulse  Calories
0        60  2020/12/01  110.0     130.0     409.1
1        60  2020/12/02  117.0     145.0     479.0
2        60  2020/12/03  103.0     135.0     340.0
3        45  2020/12/04  109.0     175.0     282.4
4        45  2020/12/05  117.0     148.0     406.0


In [8]:
# Convert the 'Date' column to datetime format
# Using errors='coerce' to handle any problematic date formats
df['Date'] = pd.to_datetime(df['Date'], errors='coerce', format=None)

In [9]:
# For any dates that failed to parse (became NaT), try alternative formats
# This handles dates without slashes like '20201214'
mask = df['Date'].isna()
if mask.any():
    print(f"\nFound {mask.sum()} dates that couldn't be parsed with standard format")
    # Try to parse those dates with a different format
    df.loc[mask, 'Date'] = pd.to_datetime(df.loc[mask, 'Date'], format='%Y%m%d', errors='coerce')


Found 127 dates that couldn't be parsed with standard format


In [10]:
# Check if any dates still couldn't be parsed
still_invalid = df['Date'].isna().sum()
if still_invalid > 0:
    print(f"\nWarning: {still_invalid} dates still couldn't be parsed!")

In [11]:
# Display the result
print("\nData types after conversion:")
print(df.dtypes)
print("\nFirst few rows after conversion:")
print(df.head())


Data types after conversion:
Duration             int64
Date        datetime64[ns]
Pulse              float64
Maxpulse           float64
Calories           float64
dtype: object

First few rows after conversion:
   Duration       Date  Pulse  Maxpulse  Calories
0        60 2020-12-01  110.0     130.0     409.1
1        60 2020-12-02  117.0     145.0     479.0
2        60 2020-12-03  103.0     135.0     340.0
3        45 2020-12-04  109.0     175.0     282.4
4        45 2020-12-05  117.0     148.0     406.0


In [12]:
# Save the modified DataFrame back to CSV
df.to_csv('test_1_calorie_datetime.csv', index=False)
print("\nSaved converted data to 'test_1_calorie_datetime.csv'")


Saved converted data to 'test_1_calorie_datetime.csv'


In [13]:
ds2 = pd.read_csv('test_1_calorie_datetime.csv')

In [14]:
ds2

,Duration,Date,Pulse,Maxpulse,Calories
0,60,2020-12-01,110.0,130.0,409.1
1,60,2020-12-02,117.0,145.0,479.0
2,60,2020-12-03,103.0,135.0,340.0
3,45,2020-12-04,109.0,175.0,282.4
4,45,2020-12-05,117.0,148.0,406.0
...,...,...,...,...,...
170,60,NaN,104.0,131.0,350.0
171,90,NaN,107.0,138.0,380.0
172,45,NaN,95.0,121.0,240.5
173,60,NaN,102.0,128.0,305.8


In [1]:
import pandas as pd
import re
from datetime import datetime

def format_date(date_str):
    """
    Convert various date formats to YYYY-MM-DD format
    """
    if pd.isna(date_str):
        return None

    # Convert integer to string
    if isinstance(date_str, (int, float)):
        date_str = str(int(date_str))

    # Clean the input
    date_str = str(date_str).strip()

    # Try various date parsing patterns
    try:
        # Case 1: Already ISO format (YYYY-MM-DD)
        if re.match(r'^\d{4}-\d{1,2}-\d{1,2}$', date_str):
            dt = datetime.strptime(date_str, '%Y-%m-%d')
            return dt.strftime('%Y-%m-%d')

        # Case 2: YYYYMMDD format (no separators)
        elif re.match(r'^\d{8}$', date_str):
            dt = datetime.strptime(date_str, '%Y%m%d')
            return dt.strftime('%Y-%m-%d')

        # Case 3: YYYY/MM/DD format
        elif re.match(r'^\d{4}/\d{1,2}/\d{1,2}$', date_str):
            dt = datetime.strptime(date_str, '%Y/%m/%d')
            return dt.strftime('%Y-%m-%d')

        # Case 4: YYYY.MM.DD format
        elif re.match(r'^\d{4}\.\d{1,2}\.\d{1,2}$', date_str):
            dt = datetime.strptime(date_str, '%Y.%m.%d')
            return dt.strftime('%Y-%m-%d')

        # Case 5: DD/MM/YYYY format
        elif re.match(r'^\d{1,2}/\d{1,2}/\d{4}$', date_str):
            dt = datetime.strptime(date_str, '%d/%m/%Y')
            return dt.strftime('%Y-%m-%d')

        # Case 6: MM/DD/YYYY format
        elif re.match(r'^\d{1,2}/\d{1,2}/\d{4}$', date_str):
            try:
                dt = datetime.strptime(date_str, '%m/%d/%Y')
                return dt.strftime('%Y-%m-%d')
            except ValueError:
                # If MM/DD/YYYY fails, it might be DD/MM/YYYY
                pass

        # Case 7: DD-MM-YYYY format
        elif re.match(r'^\d{1,2}-\d{1,2}-\d{4}$', date_str):
            dt = datetime.strptime(date_str, '%d-%m-%Y')
            return dt.strftime('%Y-%m-%d')

        # Case 8: MM-DD-YYYY format
        elif re.match(r'^\d{1,2}-\d{1,2}-\d{4}$', date_str):
            try:
                dt = datetime.strptime(date_str, '%m-%d-%Y')
                return dt.strftime('%Y-%m-%d')
            except ValueError:
                # If MM-DD-YYYY fails, it might be DD-MM-YYYY
                pass

        # Case 9: DD.MM.YYYY format
        elif re.match(r'^\d{1,2}\.\d{1,2}\.\d{4}$', date_str):
            dt = datetime.strptime(date_str, '%d.%m.%Y')
            return dt.strftime('%Y-%m-%d')

        # Case 10: Short formats like D/M/YY or M/D/YY
        elif re.match(r'^\d{1,2}/\d{1,2}/\d{2}$', date_str):
            try:
                dt = datetime.strptime(date_str, '%d/%m/%y')
                return dt.strftime('%Y-%m-%d')
            except ValueError:
                try:
                    dt = datetime.strptime(date_str, '%m/%d/%y')
                    return dt.strftime('%Y-%m-%d')
                except ValueError:
                    pass

        # Case 11: Short formats with dots D.M.YY
        elif re.match(r'^\d{1}\.\d{1,2}\.\d{2}$', date_str):
            dt = datetime.strptime(date_str, '%d.%m.%y')
            return dt.strftime('%Y-%m-%d')

        # Case 12: Short formats like DD-MM-YY or MM-DD-YY
        elif re.match(r'^\d{1,2}-\d{1,2}-\d{2}$', date_str):
            try:
                dt = datetime.strptime(date_str, '%d-%m-%y')
                return dt.strftime('%Y-%m-%d')
            except ValueError:
                try:
                    dt = datetime.strptime(date_str, '%m-%d-%y')
                    return dt.strftime('%Y-%m-%d')
                except ValueError:
                    pass

        # Case 13: Month name formats like "Jan 15 2021" or "15 Jan 2021"
        elif re.search(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)', date_str, re.IGNORECASE):
            # Try different month name formats
            formats = [
                '%b %d %Y',      # Jan 15 2021
                '%d %b %Y',      # 15 Jan 2021
                '%d-%b-%Y',      # 15-Jan-2021
                '%d-%b-%y',      # 15-Jan-21
                '%b-%d-%Y',      # Jan-15-2021
                '%b-%d-%y',      # Jan-15-21
                '%b.%d.%Y',      # Jan.15.2021
                '%d/%b/%Y',      # 15/Jan/2021
                '%b/%d/%Y',      # Jan/15/2021
                '%b/%d/%y',      # Jan/15/21
                '%Y-%b-%d',      # 2021-Jan-15
                '%d %b %y',      # 15 Jan 21
                '%b %d',         # Jan 15
                '%b %dst %Y',    # Jan 21st 2021
                '%b %dnd %Y',    # Jan 22nd 2021
                '%b %drd %Y',    # Jan 23rd 2021
                '%b %dth %Y',    # Jan 24th 2021
                '%dst %b %Y',    # 21st Jan 2021
                '%dnd %b %Y',    # 22nd Jan 2021
                '%drd %b %Y',    # 23rd Jan 2021
                '%dth %b %Y',    # 24th Jan 2021
            ]

            # Replace ordinal indicators to simplify parsing
            date_str = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', date_str)

            for fmt in formats:
                try:
                    dt = datetime.strptime(date_str, fmt)
                    return dt.strftime('%Y-%m-%d')
                except ValueError:
                    continue

        # Case 14: Full month name formats
        elif any(month in date_str for month in ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']):
            # Try different full month name formats
            formats = [
                '%B %d %Y',      # January 15 2021
                '%d %B %Y',      # 15 January 2021
                '%d-%B-%Y',      # 15-January-2021
                '%B-%d-%Y',      # January-15-2021
                '%B %dst %Y',    # January 21st 2021
                '%B %dnd %Y',    # January 22nd 2021
                '%B %drd %Y',    # January 23rd 2021
                '%B %dth %Y',    # January 24th 2021
                '%dth of %B %Y', # 24th of January 2021
                '%dst of %B %Y', # 21st of January 2021
                '%dnd of %B %Y', # 22nd of January 2021
                '%drd of %B %Y', # 23rd of January 2021
                '%d %B %y',      # 15 January 21
                '%B-%d-%y',      # January-15-21
            ]

            # Replace ordinal indicators to simplify parsing
            date_str = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', date_str)
            date_str = re.sub(r'of ', '', date_str)

            for fmt in formats:
                try:
                    dt = datetime.strptime(date_str, fmt)
                    return dt.strftime('%Y-%m-%d')
                except ValueError:
                    continue

        # Case 15: Special case for D-M-YYYY format
        elif re.match(r'^\d{1}-\d{1,2}-\d{4}$', date_str):
            dt = datetime.strptime(date_str, '%d-%m-%Y')
            return dt.strftime('%Y-%m-%d')

        # Case 16: Special case for D-M-YY format
        elif re.match(r'^\d{1}-\d{1}-\d{2}$', date_str):
            dt = datetime.strptime(date_str, '%d-%m-%y')
            return dt.strftime('%Y-%m-%d')

        # Last resort: try pandas to_datetime with errors='coerce'
        return pd.to_datetime(date_str, errors='coerce').strftime('%Y-%m-%d')

    except (ValueError, AttributeError):
        # Last resort: try pandas to_datetime with errors='coerce'
        try:
            parsed_date = pd.to_datetime(date_str, errors='coerce')
            if pd.notna(parsed_date):
                return parsed_date.strftime('%Y-%m-%d')
        except:
            pass

        # If all else fails
        return None

# Read the CSV file
df = pd.read_csv('test_1_calorie.csv')

# Display original dates sample
print("Original date samples:")
print(df['Date'].head(10))

# Apply the format_date function to the Date column
df['Original_Date'] = df['Date']  # Keep the original for reference
df['Date'] = df['Date'].apply(format_date)

# Display the results
print("\nFormatted date samples:")
print(df[['Original_Date', 'Date']].head(10))

# Count null values after formatting
null_count = df['Date'].isna().sum()
print(f"\nCount of dates that couldn't be formatted: {null_count}")

if null_count > 0:
    print("Problematic dates that couldn't be formatted:")
    problematic = df[df['Date'].isna()][['Original_Date']]
    print(problematic)

# Save to a new CSV file with consistently formatted dates
df.to_csv('test_1_calorie_formatted.csv', index=False)
print("\nSaved formatted dates to 'test_1_calorie_formatted.csv'")

# Demonstrate how pd.to_datetime can now easily convert these dates
print("\nConverting formatted dates to datetime using pd.to_datetime:")
datetime_col = pd.to_datetime(df['Date'])
print(datetime_col.head(10))
print(f"\nDatatype of converted column: {datetime_col.dtype}")

Original date samples:
0    2020/12/01
1    2020/12/02
2    2020/12/03
3    2020/12/04
4    2020/12/05
5    2020/12/06
6    2020/12/07
7    2020/12/08
8    2020/12/09
9    2020/12/10
Name: Date, dtype: object

Formatted date samples:
  Original_Date        Date
0    2020/12/01  2020-12-01
1    2020/12/02  2020-12-02
2    2020/12/03  2020-12-03
3    2020/12/04  2020-12-04
4    2020/12/05  2020-12-05
5    2020/12/06  2020-12-06
6    2020/12/07  2020-12-07
7    2020/12/08  2020-12-08
8    2020/12/09  2020-12-09
9    2020/12/10  2020-12-10

Count of dates that couldn't be formatted: 4
Problematic dates that couldn't be formatted:
    Original_Date
22            NaN
52            NaN
98        March 8
137           NaN

Saved formatted dates to 'test_1_calorie_formatted.csv'

Converting formatted dates to datetime using pd.to_datetime:
0   2020-12-01
1   2020-12-02
2   2020-12-03
3   2020-12-04
4   2020-12-05
5   2020-12-06
6   2020-12-07
7   2020-12-08
8   2020-12-09
9   2020-12-10
Name: D

In [2]:
ds3 = pd.read_csv('test_1_calorie_formatted.csv')

In [3]:
ds3

,Duration,Date,Pulse,Maxpulse,Calories,Original_Date
0,60,2020-12-01,110.0,130.0,409.1,2020/12/01
1,60,2020-12-02,117.0,145.0,479.0,2020/12/02
2,60,2020-12-03,103.0,135.0,340.0,2020/12/03
3,45,2020-12-04,109.0,175.0,282.4,2020/12/04
4,45,2020-12-05,117.0,148.0,406.0,2020/12/05
...,...,...,...,...,...,...
170,60,2021-05-18,104.0,131.0,350.0,18.05.2021
171,90,2021-05-19,107.0,138.0,380.0,19th of May 2021
172,45,2021-05-20,95.0,121.0,240.5,May-20-21
173,60,2021-05-21,102.0,128.0,305.8,21 May 21


In [4]:
ds3.dtypes

Duration           int64
Date              object
Pulse            float64
Maxpulse         float64
Calories         float64
Original_Date     object
dtype: object